# Import Libraries

In [7]:
import glob

import pandas as pd
from pytrends.request import TrendReq
from pytrends import dailydata
from datetime import date


# Import CSVs

In [8]:
df = pd.read_csv("GoogleTrends.csv")

# Using Trends

In [9]:
# Timezone Offset(US CST = 360)
pytrends = TrendReq(hl='en-US', tz=360)
weekly_dates = pd.date_range(start='2017-01-01', end='2020-12-31', freq='W')

# calculating weekly trends to scale daily trends
def calculate_trend(key):
    pytrends.build_payload([key], cat=0, timeframe='2017-01-01 2020-12-31', geo='US')
    weekly_trends = pytrends.interest_over_time()
    return weekly_trends

weekly_trend_df = pd.DataFrame(index=pd.date_range(date(2017,1,1), date(2020,12,31), freq='W'))
adjusted_df = pd.DataFrame(index=pd.date_range(date(2017,1,1), date(2020,12,31), freq='D'))

adjusted_df.index.rename('Date', inplace=True)

# Original daily trends df
for column in df:
    if not "Unnamed" in df[column].name:
        # New column for each Trend keyword - initially all values set to 0
        adjusted_df[df[column].name] = 0
        weekly_trends = calculate_trend(df[column].name)
        # using values from first column of weekly trends
        weekly_trend_df[df[column].name] = weekly_trends.iloc[:, 0]

# Traversing original daily trends df
for index, row in df.iterrows():
    row_date = row.iloc[0] # First element of series (row)
    # Get weekly trend row for which date is just before the daily trends row date
    weekly_trends_row = (weekly_trend_df.loc[weekly_trend_df.index <= row_date]).iloc[-1]
    # Adjustment handling - to be multiplies by trend value of daily trend
    weekly_trends_row_adj = weekly_trends_row.divide(100)
    new_row = row.iloc[1:].mul(weekly_trends_row_adj)
    adjusted_df.iloc[index] = new_row

adjusted_df.to_csv("GoogleTrendsAdjusted.csv", index=True)
print("Adjusted Trends Exported!")

Adjusted Trends Exported!
